# Install Conda Environments in OpenSARlab using an environment.yml

**If you wish to create a conda environment with the conda create commnand, use this [notebook](../opensarlab_docs/OpenSARlab_supplements/Jupyter_Conda_Environments.ipynb) instead.**

In [ ]:
import glob
import ipywidgets
import os
import fileinput

from IPython.display import Markdown, display

### List Currently Installed Conda Environments

In [ ]:
%conda env list

### Select a Conda Environment to Create

Note: 
to be seen by this notebook, new environment.yml files should be stored in /home/jovyan/Environment_Configs/

In [ ]:
env_ymls = glob.glob("/home/jovyan/conda_environments/Environment_Configs/*.yml")

yml_dict = {fileinput.filename():f.split('name: ')[1].strip('\n') for f in fileinput.input(env_ymls) if fileinput.isfirstline()}
names = [v for v in yml_dict.values()]
if len(names) > len(set(names)):
    dups = {}
    
    for k, v in yml_dict.items():
        dups.setdefault(v, []).append(k)
    dups = {k:v for (k,v) in dups.items() if len(dups[k]) > 1}
    display(Markdown(f'<text style=color:red><strong>WARNING:</text>'))
    display(Markdown(f'<text style=color:red>Duplicate environment names found in /home/jovyan/conda_environments/Environment_Configs/*.yml files.</text>'))
    display(Markdown(f'<text style=color:red>Creating an env with the same name as an existing env will overwrite the original.</text>'))
    display(Markdown(f'<text style=color:green><br><strong>RECOMMENDATION:</strong></text>'))
    display(Markdown(f'<text style=color:green>Edit the environment names on the first lines of the YMLs to eliminate duplicates.</text>'))   
    
    for d in dups:
        print(f'YMLs using the environment name "{d}":')
        for pth in dups[d]:
            print(pth)
    options = [f"{yml_dict[k]} ---> {k}" for k in yml_dict]
else:
    options = [yml_dict[k] for k in yml_dict]

display(Markdown(f'<text style=color:green><br><strong>Select an Environment to Create:</strong></text>'))
height = len(env_ymls) * 19
names =  ipywidgets.RadioButtons(
    options=options,
    description="",
    disabled=False,
    layout=ipywidgets.Layout(height=f"{height}px", width='1000px')
)
names


### Collect the Environment Name, Prefix, and Path to its env.yml

In [ ]:
name = names.value
if '--->' in name:
    name, yml = name.split(' ---> ')
else:
    yml = {v:k for k,v in yml_dict.items()}[name]
prefix = f"/home/jovyan/.local/envs/{name}"

### Create the Environment
This will overwrite an existing enviornment with the same name instead of updating it, which is the safer option for environments that change PATH and/or PYTHONPATH

In [ ]:
%conda env create -f $yml --prefix $prefix --force

### Update iykernel with the environment name and location of its kernel

In [ ]:
%conda run -n $name kernda --display-name $name -o $prefix/share/jupyter/kernels/python3/kernel.json

### Complete installations and repository clones for newly created insar_analysis and train environments
If you create a new env that requires an installation script for building packages from source or changing env variables, run them here. 

In [ ]:
if name == "insar_analysis":
    !source install_insar_analysis_pkgs.sh
elif name == "train":
    !source train.sh

### Remove unused packages and caches.

In [ ]:
%conda clean -p -t --yes

<font face="Calibri" size="2"> <i>Create_OSL_Conda_Environments.ipynb - Version 1.1.0 - July 2021
    <br>
        <b>Version Changes:</b>
    <ul>
        <li>Support environment.yml files following any naming convention</li>
        <li>Recognize and provide warnings for duplicate env names</li>
    </ul>
    </i>
</font>